In [11]:
## Ligand preparation

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem




filenameSDF='ExcB.sdf'
SP001smile='CCCC(=O)O[C@H]1C\C(=C/C2OC(=O)[C@@](C)(SC)[C@@]2(O)[C@@H](OC(=O)C)[C@H]3[C@@H](C)[C@@H](O)C[C@H](OC(=O)C)[C@]3(C)[C@H]1OC(=O)C)\C'

os.chdir('./Covalent_Docking')
mol=Chem.MolFromSmiles(SP001smile) 
molH=Chem.AddHs(mol)
AllChem.EmbedMolecule(molH,useRandomCoords=True,maxAttempts=500) #(molH,AllChem.ETKDG()) #(molH) #(molH,useRandomCoords=True,maxAttempts=500)
AllChem.MMFFOptimizeMoleculeConfs(molH,5000)
print ( filenameSDF, 'EmbedMolecule & MMFFOptimizeMolecule' )
print ('write SDF')
Chem.MolToMolFile(molecular,filenameSDF)

ExcB.sdf EmbedMolecule & MMFFOptimizeMolecule
write SDF


In [5]:
## Protein preparation

import re
import os, sys
from os import walk
from re import search
import subprocess
import pandas as pd
import time
import math

pdbfile='6nt5.pdb' # The protein structure of full-length hSTING for docking experiments was obtained from the Protein Data Bank (PDB) with PDB ID of 6NT5.

os.chdir('./Covalent_Docking')

pdbfilenameclean='PROclean_'+re.sub('PRO_','',pdbfile)
pdbread=open(pdbfile,'r')
MODEL=0
ENDMDL=0
pdbclean=''
for i, pdbline in enumerate (pdbread.readlines()):
    if MODEL>2:
        continue
    if re.search('^MODEL',pdbline):
        MODEL=MODEL+1
    if re.search('^ENDMDL',pdbline):
        ENDMDL=ENDMDL+1
    if MODEL<2 and ENDMDL<2:
        if (re.search('HOH',pdbline) or re.search('Na+',pdbline) or re.search('Cl-',pdbline)):
            continue
        if (re.search('ATOM',pdbline) or re.search('TER',pdbline) or re.search('END',pdbline)):
            pdbclean=pdbclean+pdbline

file_w=open(pdbfilenameclean,'w')
file_w.write(pdbclean)
file_w.close()

pdbfileamberH='PROamberH_'+re.sub('PRO_','',pdbfile)
tleapin="""source leaprc.protein.fb15
mol = loadpdb %s 
charge mol
savePdb mol %s
quit
"""%(pdbfilenameclean,pdbfileamberH)
file_w=open('tleap.inp','w')
file_w.write(tleapin)
file_w.close()


subprocess.run("tleap -f tleap.inp", shell=True, capture_output=False)

pdbread=open(pdbfileamberH,'r')
pdbclean=''
for i, pdbline in enumerate (pdbread.readlines()):
    pdbcorrect=re.sub('HET','HIS',pdbline)
    pdbcorrect=re.sub('NHIE','HIS',pdbcorrect)
    pdbcorrect=re.sub('HIE','HIS',pdbcorrect)            
    pdbcorrect=re.sub('CHIE','HIS',pdbcorrect)            
    pdbclean=pdbclean+pdbcorrect
file_w=open(pdbfileamberH,'w')
file_w.write(pdbclean)
file_w.close()

os.remove('leap.log') 
os.system('rm PROclean_* tleap.inp ')



print (pdbfileamberH)


-I: Adding /home1/drhuangwc/bin/amber20/dat/leap/prep to search path.
-I: Adding /home1/drhuangwc/bin/amber20/dat/leap/lib to search path.
-I: Adding /home1/drhuangwc/bin/amber20/dat/leap/parm to search path.
-I: Adding /home1/drhuangwc/bin/amber20/dat/leap/cmd to search path.
-f: Source tleap.inp.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.inp
----- Source: /home1/drhuangwc/bin/amber20/dat/leap/cmd/leaprc.protein.fb15
----- Source of /home1/drhuangwc/bin/amber20/dat/leap/cmd/leaprc.protein.fb15 done
Log file: ./leap.log
Loading parameters: /home1/drhuangwc/bin/amber20/dat/leap/parm/parm99.dat
Reading title:
PARM99 for DNA,RNA,AA, organic molecules, Polariz.& LP incl.02/04/99
Loading parameters: /home1/drhuangwc/bin/amber20/dat/leap/parm/frcmod.fb15
Reading force field modification type file (frcmod)
Reading title:
AMBER-FB15 parameter file.  Includes new atom types for amino acid side chains and new valence / torsion parameters.  See Wang et al., J Phys Chem B. (20